## Unpaired Image-to-Image Translation using Cycle-Consistent Adversarial Networks

In [1]:
import os
import time

import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from IPython.display import clear_output
from tensorflow_examples.tensorflow_examples.models.pix2pix import pix2pix

In [2]:
tfds.disable_progress_bar()
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [ ]:
dataset, metadata = tfds.load('cycle_gan/monet2photo',
                              with_info = True, as_supervised = True)

train_monet, train_photo = dataset['trainA'], dataset['trainB']
test_monet, test_photo = dataset['testA'], dataset['testB']

/home/tanmay/anaconda3/lib/python3.7/site-packages/urllib3/connectionpool.py:1004: InsecureRequestWarning: Unverified HTTPS request is being made to host 'people.eecs.berkeley.edu'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
